Imports des utilitaires pour le nettoyages de données

In [2]:
import pandas as pd
import pandas as pd
import numpy as np
import cv2

import os

import random

from datetime import datetime

import matplotlib.pyplot as plt

import seaborn as sns

%matplotlib inline

sns.set_theme (style = "whitegrid", palette= "pastel")

Imports des données

In [3]:
test = pd.read_csv("X_test_update.csv", sep = ',')
train = pd.read_csv("X_train_update.csv", sep = ',')

In [4]:
# SUPPRESSION DES DOUBLONS EN COLONNE

# Avant de concaténer Designation et Description, on vérifie qu'il n'y a pas la même chose dans ces deux colonnes
nombre_lignes = (train["designation"] == train["description"]).sum()
print("Il y a",nombre_lignes,"lignes où designation et description contiennent les mêmes informations")

# On remplace alors la description par une chaine de caractère vide pour préparer la concaténation
train.loc[train["designation"] == train["description"], "description"] = ""

Il y a 179 lignes où designation et description contiennent les mêmes informations


Merge des colonnes designation et description dans une nouvelles colonnes "merges"

In [5]:
train_merge = train

# Fusion des colonnes 'designation' et 'description'
train_merge['merged'] = pd.concat([
    train_merge['designation'].fillna(''), 
    train_merge['description'].fillna('')
], axis=1).apply(' '.join, axis=1)

# Nettoyage des espaces inutiles dans la colonne fusionnée
train_merge['merged'] = train_merge['merged'].str.strip()

# Suppression des doublons en fonction de la colonne fusionnée
train_merge = train_merge.drop_duplicates(subset=['merged'])

train_merge= train_merge.drop(columns=['designation','description', "Unnamed: 0"])

# Résultat final
print(train_merge)

        productid     imageid  \
0      3804725264  1263597046   
1       436067568  1008141237   
2       201115110   938777978   
3        50418756   457047496   
4       278535884  1077757786   
...           ...         ...   
84911   206719094   941495734   
84912  3065095706  1188462883   
84913   440707564  1009325617   
84914  3942400296  1267353403   
84915    57203227   684671297   

                                                  merged  
0      Olivia: Personalisiertes Notizbuch / 150 Seite...  
1      Journal Des Arts (Le) N° 133 Du 28/09/2001 - L...  
2      Grand Stylet Ergonomique Bleu Gamepad Nintendo...  
3      Peluche Donald - Europe - Disneyland 2000 (Mar...  
4      La Guerre Des Tuques Luc a des id&eacute;es de...  
...                                                  ...  
84911                        The Sims [ Import Anglais ]  
84912  Kit piscine acier NEVADA déco pierre Ø 3.50m x...  
84913  Journal Officiel De La Republique Francaise N°...  
84914  Table 

On vérifie ensuite s'il existe des doublons

In [6]:
# Vérifier si des doublons existent dans la colonne 'merged'
duplicates_exist = train_merge['merged'].duplicated().any()

# Afficher le résultat
if duplicates_exist:
    print("Il existe des doublons dans la colonne 'merged'.")
    # Afficher les lignes dupliquées
    print("Voici les doublons :")
    print(train_merge[train_merge['merged'].duplicated(keep=False)])
else:
    print("Aucun doublon trouvé dans la colonne 'merged'.")

Aucun doublon trouvé dans la colonne 'merged'.


Dans la colonnes "merged" on regarde la fréquence des mots 

In [7]:
from collections import Counter
import re

# Concaténer toutes les lignes de la colonne 'merged' dans une seule chaîne
all_text = " ".join(train_merge['merged'].dropna())

# Nettoyer le texte (enlever les caractères spéciaux, mettre en minuscule, etc.)
cleaned_text = re.sub(r"[^a-zA-Z0-9\s]", "", all_text.lower())

# Diviser le texte en mots
words = cleaned_text.split()

# Calculer la fréquence des mots
word_counts = Counter(words)

# Afficher les 10 mots les plus fréquents
print("Les 10 mots les plus fréquents :")
for word, count in word_counts.most_common(10):
    print(f"{word}: {count}")

# Convertir en DataFrame pour une visualisation tabulaire si nécessaire
word_counts_df = pd.DataFrame(word_counts.items(), columns=['Word', 'Frequency']).sort_values(by='Frequency', ascending=False)

# Afficher les 10 premiers mots
print(word_counts_df.head(10))


Les 10 mots les plus fréquents :
de: 400617
la: 152722
et: 149260
pour: 96356
en: 91686
le: 89042
les: 77115
x: 73618
des: 54567
un: 50470
     Word  Frequency
55     de     400617
31     la     152722
19     et     149260
59   pour      96356
105    en      91686
13     le      89042
239   les      77115
167     x      73618
11    des      54567
61     un      50470


In [10]:
# NORMALISATION DU TEXTE

from unidecode import unidecode

def clean_text(text):
    if isinstance(text, str):            # Si le texte est une chaîne de caractères,
        text = unidecode(text)             # Remplace les accents par des lettres sans accent
        text = text.lower()                # Met le texte en minuscule
        text = ' '.join(text.split())      # Remplace les espaces inutiles
        return text
    return text

train_merge["merged"] = train_merge['merged'].apply(clean_text)
display(train_merge.head(40))

,productid,imageid,merged
0,3804725264,1263597046,olivia: personalisiertes notizbuch / 150 seite...
1,436067568,1008141237,journal des arts (le) ndeg 133 du 28/09/2001 -...
2,201115110,938777978,grand stylet ergonomique bleu gamepad nintendo...
3,50418756,457047496,peluche donald - europe - disneyland 2000 (mar...
4,278535884,1077757786,la guerre des tuques luc a des id&eacute;es de...
5,5862738,393356830,afrique contemporaine ndeg 212 hiver 2004 - do...
6,91920807,907794536,christof e: bildungsprozessen auf der spur
7,344240059,999581347,conquerant sept cahier couverture polypro 240 ...
8,4239126071,1325918866,puzzle scooby-doo avec poster 2x35 pieces
9,3793572222,1245644185,tente pliante v3s5-pro pvc blanc - 3 x 4m50 - ...


In [11]:
# NOUVEAU CONTRÔLE SUR LES DOUBLONS

nb_lignes_avant = len(train_merge)
train_merge = train_merge.drop_duplicates(subset=["merged"])                 # On supprime les doublons en ne gardant que la première occurance 

nb_lignes_apres = len(train_merge)
print("Nouveaux doublons supprimés :", nb_lignes_avant - nb_lignes_apres)
print("Il reste :",nb_lignes_apres,"lignes\n")

Nouveaux doublons supprimés : 38
Il reste : 83463 lignes



In [12]:
# TRAITEMENT DES BALISES HTML (type <b>, </p>, etc)

# Importation de la bibliothèque "Regular Expression"
import re 


# Fonction qui identifie tout ce qui est délimité par < >
def extraction_balises(texte):
    return re.findall(r"<[^>]+>", str(texte))     


# Création d'une liste d'éléments uniques
balises = set()                                   
train_merge["merged"].apply(lambda x: balises.update(extraction_balises(x)))


# Création d'un dataframe et export Excel pour meilleur visibilité
df_balises = pd.DataFrame(list(balises), columns=["Balises"])
df_balises["Balises"].to_excel("fichier_balises.xlsx", index=False)

print("Nous avons identifier",len(df_balises),"balises uniques")

Nous avons identifier 984 balises uniques


In [13]:
# SUPPRESSION DES DOUBLES CHEVRONS
train_merge["merged"] = train_merge["merged"].str.replace("<<", "").str.replace(">>", "")

In [14]:
# On relance le traitement des balises HTML

balises = set()                                   
train_merge["merged"].apply(lambda x: balises.update(extraction_balises(x)))

df_balises = pd.DataFrame(list(balises), columns=["Balises"])
df_balises["Balises"].to_excel("fichier_balises.xlsx", index=False)

print("Il ne reste plus que",len(df_balises),"balises uniques après suppression des doubles chevrons délimitants du texte")

Il ne reste plus que 317 balises uniques après suppression des doubles chevrons délimitants du texte


In [15]:
df_html = train_merge["merged"].str.extract(r"(&\s\w+)")
df_html = df_html.dropna()
df_html.columns = ['caractere']

display(df_html.info())
print("\n")
display(df_html.head(10))
print("\n")
display(df_html.value_counts().head(30))

<class 'pandas.core.frame.DataFrame'>
Index: 1350 entries, 62 to 84879
Data columns (total 1 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   caractere  1350 non-null   object
dtypes: object(1)
memory usage: 21.1+ KB


None

,caractere
62,& creme
74,& 2
96,& son
196,& r2
327,& bunny
335,& engineering
385,& mortimer
416,& hal
417,& 2
446,& gris


caractere 
& 2           63
& blanc       31
& dragons     31
& the         20
& 3           19
& decker      15
& d           13
& smart       12
& ntsc        12
& triphase    11
& clear       10
& noir         9
& zoom         9
& mavic        8
& points       8
& cadre        8
& cie          8
& vie          7
& spice        7
& stratton     7
& leopard      7
& piano        7
& 4            7
& magic        6
& mrs          6
& la           6
& le           6
& play         6
& carreaux     6
& girl         6
Name: count, dtype: int64

In [16]:
import pandas as pd

# Supposons que votre DataFrame soit chargé dans une variable appelée df
# df = pd.read_csv('votre_fichier.csv')

# 1. Vérifier les valeurs manquantes
print("Valeurs manquantes avant nettoyage :")
print(train_merge.isnull().sum())

# 2. Supprimer les lignes avec des valeurs manquantes (si nécessaire)
train_merge = train_merge.dropna()

# 3. Supprimer les doublons
train_merge = train_merge.drop_duplicates()

# 4. Normaliser les données (par exemple, convertir toutes les chaînes en minuscules)
train_merge['merged'] = train_merge['merged'].str.lower()

# 5. Vérifier les types de données
print("Types de données après nettoyage :")
print(train_merge.dtypes)

# Afficher les premières lignes du DataFrame nettoyé
print("DataFrame nettoyé :")
print(train_merge.head())


Valeurs manquantes avant nettoyage :
productid    0
imageid      0
merged       0
dtype: int64
Types de données après nettoyage :
productid     int64
imageid       int64
merged       object
dtype: object
DataFrame nettoyé :
    productid     imageid                                             merged
0  3804725264  1263597046  olivia: personalisiertes notizbuch / 150 seite...
1   436067568  1008141237  journal des arts (le) ndeg 133 du 28/09/2001 -...
2   201115110   938777978  grand stylet ergonomique bleu gamepad nintendo...
3    50418756   457047496  peluche donald - europe - disneyland 2000 (mar...
4   278535884  1077757786  la guerre des tuques luc a des id&eacute;es de...


In [17]:
import pandas as pd
import html

unique_values = train_merge['merged'].unique()
print("Valeurs uniques dans la colonne 'merged' :")
print(unique_values[:10])  # Afficher les 10 premières valeurs uniques

# 2. Nettoyer les caractères spéciaux et les entités HTML
train_merge['merged'] = train_merge['merged'].apply(lambda x: html.unescape(x))
train_merge['merged'] = train_merge['merged'].str.replace(r'[^\w\s]', '', regex=True)

# 3. Vérifier les doublons après normalisation
train_merge = train_merge.drop_duplicates()

# Afficher les premières lignes du DataFrame nettoyé
print("DataFrame nettoyé après normalisation :")
print(train_merge.head())

Valeurs uniques dans la colonne 'merged' :
['olivia: personalisiertes notizbuch / 150 seiten / punktraster / ca din a5 / rosen-design'
 "journal des arts (le) ndeg 133 du 28/09/2001 - l'art et son marche salon d'art asiatique a paris - jacques barrere - francois perrier - la reforme des ventes aux encheres publiques - le sna fete ses cent ans."
 'grand stylet ergonomique bleu gamepad nintendo wii u - speedlink pilot style pilot style touch pen de marque speedlink est 1 stylet ergonomique pour gamepad nintendo wii u.<br> pour un confort optimal et une precision maximale sur le gamepad de la wii u: ce grand stylet hautement ergonomique est non seulement parfaitement adapte a votre main mais aussi tres elegant.<br> il est livre avec un support qui se fixe sans adhesif a l\'arriere du gamepad<br> <br> caracteristiques:<br> modele: speedlink pilot style touch pen<br> couleur: bleu<br> ref. fabricant: sl-3468-be<br> compatibilite: gamepad nintendo wii u<br> forme particulierement ergonomique

In [ ]:
import pandas as pd
import html
import re


# Fonction pour nettoyer les descriptions
def clean_description(text):
    # Supprimer les balises HTML
    text = re.sub(r'<.*?>', '', text)
    # Convertir les entités HTML en caractères normaux
    text = html.unescape(text)
    # Supprimer les caractères spéciaux
    text = re.sub(r'[^\w\s]', '', text)
    # Normaliser les espaces
    text = re.sub(r'\s+', ' ', text).strip()
    return text

# Appliquer la fonction de nettoyage à la colonne 'merged'
train_merge['merged'] = train_merge['merged'].apply(clean_description)

# Vérifier les doublons après normalisation
train_merge = train_merge.drop_duplicates()

# Afficher les premières lignes du DataFrame nettoyé
print("DataFrame nettoyé après normalisation :")
print(train_merge.head(40))

DataFrame nettoyé après normalisation :
     productid     imageid                                             merged
0   3804725264  1263597046  olivia personalisiertes notizbuch 150 seiten p...
1    436067568  1008141237  journal des arts le ndeg 133 du 28092001 lart ...
2    201115110   938777978  grand stylet ergonomique bleu gamepad nintendo...
3     50418756   457047496  peluche donald europe disneyland 2000 marionne...
4    278535884  1077757786  la guerre des tuques luc a des idées de grande...
5      5862738   393356830  afrique contemporaine ndeg 212 hiver 2004 doss...
6     91920807   907794536          christof e bildungsprozessen auf der spur
7    344240059   999581347  conquerant sept cahier couverture polypro 240 ...
8   4239126071  1325918866           puzzle scoobydoo avec poster 2x35 pieces
9   3793572222  1245644185  tente pliante v3s5pro pvc blanc 3 x 4m50 longu...
10  1915836983  1111840281  eames inspired sxw chair pink black the timele...
11  4127967621  12958169

In [19]:
import pandas as pd

# Supposons que votre DataFrame soit chargé dans une variable appelée df
# df = pd.read_csv('votre_fichier.csv')

# 1. Vérifier les types de données
print("Types de données après nettoyage :")
print(train_merge.dtypes)

# 2. Vérifier les valeurs uniques dans la colonne 'merged' après nettoyage
unique_values = train_merge['merged'].unique()
print("Valeurs uniques dans la colonne 'merged' après nettoyage :")
print(unique_values[:10])  # Afficher les 10 premières valeurs uniques

# 3. Sauvegarder le DataFrame nettoyé
train_merge.to_csv('dataframe_nettoye.csv', index=False)

# Afficher les premières lignes du DataFrame nettoyé
print("DataFrame nettoyé final :")
print(train_merge.head())


Types de données après nettoyage :
productid     int64
imageid       int64
merged       object
dtype: object


Valeurs uniques dans la colonne 'merged' après nettoyage :
['olivia personalisiertes notizbuch 150 seiten punktraster ca din a5 rosendesign'
 'journal des arts le ndeg 133 du 28092001 lart et son marche salon dart asiatique a paris jacques barrere francois perrier la reforme des ventes aux encheres publiques le sna fete ses cent ans'
 'grand stylet ergonomique bleu gamepad nintendo wii u speedlink pilot style pilot style touch pen de marque speedlink est 1 stylet ergonomique pour gamepad nintendo wii ubr pour un confort optimal et une precision maximale sur le gamepad de la wii u ce grand stylet hautement ergonomique est non seulement parfaitement adapte a votre main mais aussi tres elegantbr il est livre avec un support qui se fixe sans adhesif a larriere du gamepadbr br caracteristiquesbr modele speedlink pilot style touch penbr couleur bleubr ref fabricant sl3468bebr compatibilite gamepad nintendo wii ubr forme particulierement ergonomique excellente tenue en mainbr pointe a revetem

In [21]:
import pandas as pd
import re

# Fonction pour segmenter les mots collés
def segment_words(text):
    # Ajouter des espaces entre les mots collés
    text = re.sub(r'([a-z])([A-Z])', r'\1 \2', text)
    text = re.sub(r'(\d)([a-zA-Z])', r'\1 \2', text)
    text = re.sub(r'([a-zA-Z])(\d)', r'\1 \2', text)
    return text

# Fonction pour corriger l'orthographe (exemple simple, vous pouvez utiliser une bibliothèque plus avancée)
def correct_spelling(text):
    # Exemple de correction simple, vous pouvez utiliser une bibliothèque comme pyspellchecker pour plus de précision
    corrections = {
        'personalisiertes': 'personalisiertes',
        'notizbuch': 'Notizbuch',
        'seiten': 'Seiten',
        'punktraster': 'Punktraster',
        'din': 'DIN',
        'rosendesign': 'Rosendesign'
    }
    words = text.split()
    corrected_words = [corrections.get(word, word) for word in words]
    return ' '.join(corrected_words)


# Supposons que votre DataFrame soit chargé dans une variable appelée df
# df = pd.read_csv('votre_fichier.csv')

# Appliquer les fonctions de segmentation, correction et traduction à la colonne 'merged'
train_merge['merged_segmented'] = train_merge['merged'].apply(segment_words)
train_merge['merged_corrected'] = train_merge['merged_segmented'].apply(correct_spelling)

# Afficher les premières lignes du DataFrame avec les descriptions traduites et corrigées
print("DataFrame avec descriptions traduites et corrigées :")
print(train_merge.head())

# Sauvegarder le DataFrame avec les descriptions traduites et corrigées
train_merge.to_csv('dataframe_traduit_et_corrige.csv', index=False)


DataFrame avec descriptions traduites et corrigées :
    productid     imageid                                             merged  \
0  3804725264  1263597046  olivia personalisiertes notizbuch 150 seiten p...   
1   436067568  1008141237  journal des arts le ndeg 133 du 28092001 lart ...   
2   201115110   938777978  grand stylet ergonomique bleu gamepad nintendo...   
3    50418756   457047496  peluche donald europe disneyland 2000 marionne...   
4   278535884  1077757786  la guerre des tuques luc a des idées de grande...   

                                    merged_segmented  \
0  olivia personalisiertes notizbuch 150 seiten p...   
1  journal des arts le ndeg 133 du 28092001 lart ...   
2  grand stylet ergonomique bleu gamepad nintendo...   
3  peluche donald europe disneyland 2000 marionne...   
4  la guerre des tuques luc a des idées de grande...   

                                    merged_corrected  
0  olivia personalisiertes Notizbuch 150 Seiten P...  
1  journal des arts

In [ ]:
import pandas as pd
import re
from spellchecker import SpellChecker
from joblib import Parallel, delayed
from tqdm import tqdm

# Initialiser le correcteur orthographique pour le français
spell = SpellChecker(language='fr')

# Fonction pour segmenter les mots collés
def segment_words(text):
    if not isinstance(text, str):
        return ""
    # Ajouter des espaces entre les mots collés
    text = re.sub(r'([a-z])([A-Z])', r'\1 \2', text)
    text = re.sub(r'(\d)([a-zA-Z])', r'\1 \2', text)
    text = re.sub(r'([a-zA-Z])(\d)', r'\1 \2', text)
    return text

# Fonction pour corriger l'orthographe
def correct_spelling(text):
    if not isinstance(text, str):
        return ""
    words = text.split()
    corrected_words = [spell.correction(word) or word for word in words]
    return ' '.join(corrected_words)

# Fonction pour traiter un lot
def process_batch(batch):
    try:
        batch['merged_segmented'] = batch['merged'].apply(segment_words)
        batch['merged_corrected'] = batch['merged_segmented'].apply(correct_spelling)
    except Exception as e:
        print(f"Error processing batch: {e}")
    return batch

# Supposons que votre DataFrame soit chargé dans une variable appelée train_merge
# train_merge = pd.read_csv('votre_fichier.csv')

# Convertir la colonne 'merged' en chaînes de caractères
train_merge['merged'] = train_merge['merged'].astype(str)

# Traitement par lots avec parallélisation et affichage de l'avancement
batch_size = 500  # Réduire la taille des lots pour éviter les problèmes de mémoire
num_batches = len(train_merge) // batch_size + 1
batches = [train_merge[i*batch_size:(i+1)*batch_size].copy() for i in range(num_batches)]

# Utiliser tqdm pour afficher l'avancement
processed_batches = Parallel(n_jobs=-1)(delayed(process_batch)(batch) for batch in tqdm(batches, desc="Processing batches", total=num_batches))

# Concaténer les lots traités
train_merge = pd.concat(processed_batches)

# Afficher les premières lignes du DataFrame avec les descriptions corrigées
print("DataFrame avec descriptions corrigées :")
print(train_merge.head())

# Sauvegarder le DataFrame avec les descriptions corrigées
train_merge.to_csv('dataframe_corrige.csv', index=False)



KeyboardInterrupt: 

detecter la langue,
Traduire,
Faire le nettoyage des lignes qui ont enormement de caractere (majorité - 4000 certaines a 12000. Faire la moyenne et couper la ligne (tronquer))
Equilibre de classes (a voir avec le mentor)
